In [1]:
import pandas as pd
import numpy as np
import os, sys
from datetime import datetime
from pprint import pprint
import numpy as np
import tqdm
# Gensim
import gensim
import gensim.corpora as corpora
from gensim.models import CoherenceModel
# For Visualization
import pyLDAvis.gensim
import pickle 
import pyLDAvis

In [2]:
df = pd.read_pickle('uniqueData13mill')

In [ ]:
# df['TextString'] = [' '.join(map(str, l)) for l in df['text']]
# df = df.drop_duplicates(subset=['TextString'])

In [ ]:
# df = df.drop('TextString', 1)

In [3]:
# Set up log to external log file
import logging
logging.basicConfig(filename='lda_13mill_model.log', format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

In [4]:
df

,hashtags,text
0,NaN,"[gotta, love, fact]"
1,NaN,"[great, day, action, message, dear, dan, toola..."
2,NaN,"[harvey, norman, reckons, climate, change, bun..."
3,NaN,"[skepticism, immigration, walk, hand, hand, sk..."
4,climatechange,"[th, november, creative, citizen, w, practical..."
...,...,...
700281,science scicomm climatechange,"[year, arctic, ice, receded, it, lowest, level..."
700284,risingseas climatechange,"[mum, mum, water, here, coastal, saintlouis, s..."
700285,workshop SDG climate BUP ClimateChange Environ...,"[day, long, action, held]"
700287,NaN,"[climate, change, lucifer, heatwave, far, like..."


In [5]:
texts = df['text'].tolist()

In [6]:
# Create Dictionary
id2word = corpora.Dictionary(texts)

In [7]:
# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [8]:
# View
print(corpus[:1])

[[(0, 1), (1, 1), (2, 1)]]


In [9]:
%%time
# Build LDA model
lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                       id2word=id2word,
                                       num_topics=15, 
                                       random_state=100,
                                       chunksize=100,
                                       passes=2,
                                       workers=7,
                                       per_word_topics=True)

CPU times: user 15h 32min 22s, sys: 4h 56min 27s, total: 20h 28min 49s
Wall time: 19h 17min 58s


In [10]:
# Print the Keyword in the 15 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.224*"global" + 0.196*"warming" + 0.032*"weather" + 0.014*"extreme" + '
  '0.013*"year" + 0.013*"temperature" + 0.012*"c" + 0.011*"it" + 0.011*"day" + '
  '0.010*"like"'),
 (1,
  '0.040*"fuel" + 0.031*"fossil" + 0.025*"food" + 0.019*"america" + 0.018*"uk" '
  '+ 0.016*"sustainable" + 0.016*"development" + 0.015*"security" + '
  '0.013*"conference" + 0.013*"history"'),
 (2,
  '0.060*"climate" + 0.058*"change" + 0.031*"it" + 0.024*"people" + '
  '0.019*"like" + 0.019*"human" + 0.015*"need" + 0.015*"time" + 0.013*"right" '
  '+ 0.012*"let"'),
 (3,
  '0.032*"im" + 0.028*"thanks" + 0.027*"you" + 0.025*"question" + 0.023*"love" '
  '+ 0.022*"hope" + 0.020*"thank" + 0.020*"check" + 0.017*"matter" + '
  '0.016*"read"'),
 (4,
  '0.044*"ice" + 0.038*"threat" + 0.038*"water" + 0.032*"community" + '
  '0.030*"care" + 0.028*"planet" + 0.025*"earth" + 0.022*"animal" + '
  '0.019*"biggest" + 0.018*"island"'),
 (5,
  '0.059*"new" + 0.023*"city" + 0.020*"sea" + 0.019*"year" + 0.016*"level" + '

In [11]:
# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.35544951415831677


In [12]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))


Perplexity:  -13.168933297677668


In [13]:
import pyLDAvis.gensim
import pickle 
import pyLDAvis
# Visualize the topics
pyLDAvis.enable_notebook()
LDAvis_prepared = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
LDAvis_prepared

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.285244  0.039422       1        1  13.698088
10    -0.286324 -0.054623       2        1  12.941586
11    -0.294534  0.082520       3        1  10.009440
12    -0.044678 -0.285932       4        1   9.167267
0     -0.082233  0.101319       5        1   7.519437
5     -0.002437 -0.211289       6        1   7.508721
6     -0.002802  0.062206       7        1   6.022279
13     0.022300  0.152676       8        1   5.064663
3      0.093775  0.138170       9        1   4.385983
1      0.148798 -0.064766      10        1   4.314849
7      0.178275  0.121973      11        1   4.222852
4      0.098810 -0.009656      12        1   4.129241
14     0.157013 -0.142616      13        1   3.864640
8      0.154111 -0.027313      14        1   3.613308
9      0.145167  0.097909      15        1   3.537638, topic_info=         Term          Freq         Total Category  logprob  loglift
19    climate  1.979044e+06  1.979044e+06  Default  30.0000  30.0000
18     change  1.843758e+06  1.843758e+06  Default  29.0000  29.0000
120    global  6.905780e+05  6.905780e+05  Default  28.0000  28.0000
125   warming  6.016820e+05  6.016820e+05  Default  27.0000  27.0000
4         amp  3.652600e+05  3.652600e+05  Default  26.0000  26.0000
...       ...           ...           ...      ...      ...      ...
698      john  1.303806e+04  1.303933e+04  Topic15  -4.7098   3.3416
1657  plastic  1.744132e+04  1.979438e+04  Topic15  -4.4188   3.2152
465     story  2.499507e+04  3.794257e+04  Topic15  -4.0590   2.9243
1547     loss  1.788526e+04  2.817365e+04  Topic15  -4.3937   2.8873
292      here  1.512355e+04  6.126680e+04  Topic15  -4.5614   1.9427

[690 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
1890      3  0.999909          2
522      12  0.999905       able
3283      8  0.999866      about
2406      6  0.999937  according
3         1  0.264726     action
...     ...       ...        ...
339       3  0.999976      youre
2523      2  0.999908      youth
1363      8  0.999881      youve
3485     11  0.999949         yr
1238     13  0.999898       zero

[922 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 11, 12, 13, 1, 6, 7, 14, 4, 2, 8, 5, 15, 9, 10])

In [ ]:
# supporting function
def compute_coherence_values(corpus, dictionary, k, a, b):
    
    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b,
                                           per_word_topics=True)
    
    coherence_model_lda = CoherenceModel(model=lda_model, texts=texts, dictionary=id2word, coherence='c_v')
    
    return coherence_model_lda.get_coherence()

In [ ]:
# Hyperparamter Tuning
grid = {}
grid['Validation_Set'] = {}

# Topics range
min_topics = 10
max_topics = 22
step_size = 3
topics_range = range(min_topics, max_topics, step_size)

# Alpha parameter
alpha = list(np.arange(0.01, 1, 0.3))
alpha.append('symmetric')
alpha.append('asymmetric')

# Beta parameter
beta = list(np.arange(0.01, 1, 0.3))
beta.append('symmetric')

# Validation sets
num_of_docs = len(corpus)
corpus_sets = [# gensim.utils.ClippedCorpus(corpus, num_of_docs*0.25), 
               # gensim.utils.ClippedCorpus(corpus, num_of_docs*0.5), 
               gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)), 
               corpus]
corpus_title = ['75% Corpus', '100% Corpus']
model_results = {'Validation_Set': [],
                 'Topics': [],
                 'Alpha': [],
                 'Beta': [],
                 'Coherence': []
                }

# Can take a long time to run
if 1 == 1:
    pbar = tqdm.tqdm(total=540)
    
    # iterate through validation corpuses
    for i in range(len(corpus_sets)):
        # iterate through number of topics
        for k in topics_range:
            # iterate through alpha values
            for a in alpha:
                # iterare through beta values
                for b in beta:
                    # get the coherence score for the given parameters
                    cv = compute_coherence_values(corpus=corpus_sets[i], dictionary=id2word, 
                                                  k=k, a=a, b=b)
                    # Save the model results
                    model_results['Validation_Set'].append(corpus_title[i])
                    model_results['Topics'].append(k)
                    model_results['Alpha'].append(a)
                    model_results['Beta'].append(b)
                    model_results['Coherence'].append(cv)
                    
                    pbar.update(1)
    pd.DataFrame(model_results).to_csv('lda_tuning_results.csv', index=False)
    pbar.close()

In [14]:
def format_topics_sentences(ldamodel=None, corpus=corpus, texts=texts):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row_list in enumerate(ldamodel[corpus]):
        row = row_list[0] if ldamodel.per_word_topics else row_list            
        # print(row)
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)

In [ ]:
df_topic_sents_keywords = format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=texts)

# Hashtags
hashtags = df['hashtags']
hashtags = hashtags.reset_index()

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']
df_dominant_topic.head(10)

# Merge
df_with_hashtags = df_dominant_topic.join(hashtags)
df_with_hashtags = df_with_hashtags.drop('index', 1)

In [ ]:
df_with_hashtags

In [ ]:
import pickle

In [ ]:
lda_model.save('lda13mill.model')

In [ ]:
# # To Load again later
# # later on, load trained model from file
# model =  models.LdaModel.load('lda.model')

In [ ]:
# save the model to disk
filename = '13Million_model.sav'
pickle.dump(lda_model, open(filename, 'wb'))

In [ ]:
# # To Load again later
# # load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)
# print(result)